In [9]:
import torch.nn as nn
import torch
import torch.nn.functional as F

import torch
import torch.nn as nn
import torchmetrics

from tqdm import tqdm 
import hydra
# from omegaconf import DictConfig

from clearml import Task, Logger

from datamodule import data, utils
from models import LSTM, CNN, Bertweet, Roberta

from transformers import BertModel
bert = BertModel.from_pretrained('bert-base-uncased')

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
DEVICE = device

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
tokens = tokenizer("in desperate need of and i can not stress this enough spring break", padding='max_length', max_length=12, truncation=True, return_tensors='pt')  
tokens

{'input_ids': tensor([[ 101, 1999, 7143, 2342, 1997, 1998, 1045, 2064, 2025, 6911, 2023,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [20]:

class BERTSentiment(nn.Module):
    def __init__(self,
                 bert,
                 output_dim):
        super(BERTSentiment, self).__init__()

        self.bert = bert
        embedding_dim = bert.config.to_dict()['hidden_size']
        self.LSTM = torch.nn.LSTM(768, 384, batch_first=True, bidirectional=True)
        self.out = nn.Linear(768, output_dim)

    def forward(self, text, mask):
        #text = [batch size, sent len]
        # seq_layers, pooled_output = self.bert(text)
        embedded = self.bert(text, mask)[0]

        # embedded = embedded[1]
        lstm_output, (last_hidden, _) = self.LSTM(embedded)
        #embedded = [batch size, emb dim]

        output_hidden = torch.cat((last_hidden[0], last_hidden[1]), dim=-1)
        output_hidden = F.dropout(output_hidden,0.2)
        
        # output_hidden = F.dropout(embedded, 0.2)
        output = self.out(output_hidden)

        #output = [batch size, out dim]
        return output

        
model = BERTSentiment(bert, 2).to(device)

In [26]:
# print(model(tokens.input_ids, tokens.attention_mask))
input_ids = tokens.input_ids.to(device)
mask = tokens.attention_mask.to(device)
print(model(input_ids, mask))

ClearML Task: created new task id=aff2ccdbba2e4a31ad9235bcdfc43e1d
ClearML new version available: upgrade to v1.3.2 is recommended!
ClearML results page: https://app.clear.ml/projects/dbb746eee2fa4a9485a64c341e357772/experiments/aff2ccdbba2e4a31ad9235bcdfc43e1d/output/log
tensor([[ 0.9134, -0.9969]], device='cuda:0', grad_fn=<AddmmBackward0>)
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [22]:
SEED = 97
sarc_path = '/home/tegzes/Desktop/FL-Detection-Experiments/datamodule/isarcasm2022.csv'
BATCH_SIZE_TRAIN = 1
BATCH_SIZE_TEST = 1
MAX_LEN = 256
HIDDEN_DIM = 64
OUTPUT_DIM = 1
EMBEDDING_LENGTH = 300
N_LAYERS = 2
LEARNING_RATE = 1e-5
BIDIRECTIONAL = False
DROPOUT = 0.25
N_EPOCHS = 3
utils.seed_everything(SEED)


In [23]:
train_iterator, valid_iterator, test_iterator = data.roberta_data_loader(sarc_path, BATCH_SIZE_TRAIN, BATCH_SIZE_TEST, True, 0, MAX_LEN, tokenizer, SEED)


In [24]:
cross_entropy_loss = nn.CrossEntropyLoss()
bce_loss = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct



In [27]:
task = Task.init(project_name="FL Detection", task_name="Training")

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    no_of_iterations = 0
    no_of_examples = 0
    
    # using torch metrics to calculate the metrics
    metric_acc = torchmetrics.Accuracy().to(torch.device("cuda", 0))
    metric_f1 = torchmetrics.F1(num_classes = 2, average="none").to(torch.device("cuda", 0))
    metric_f1_micro = torchmetrics.F1(num_classes = 2).to(torch.device("cuda", 0))
    metric_f1_macro = torchmetrics.F1(num_classes = 2, average='macro').to(torch.device("cuda", 0))
    metric_precision = torchmetrics.Precision(num_classes = 2, average="none").to(torch.device("cuda", 0))
    metric_recall = torchmetrics.Recall(num_classes = 2, average="none").to(torch.device("cuda", 0))
  
    
    model.train()

    for batch_idx, batch in tqdm(enumerate(iterator, 0)):
        
        ids = batch['ids'].to(DEVICE, dtype = torch.long)
        mask = batch['mask'].to(DEVICE, dtype = torch.long)
        token_type_ids = batch['token_type_ids'].to(DEVICE, dtype = torch.long)
        targets = batch['targets'].to(DEVICE, dtype = torch.long)
        # tweet_lens = batch['tweet_len']

        outputs = model(ids, mask)#, tweet_lens)#.to('cpu'))
        # outputs = model(ids, mask, token_type_ids)

        # targets = targets.unsqueeze(1) # for BCEWithLogitsLoss criterion
        loss = criterion(outputs, targets)
        epoch_loss += loss.item()

        _, predictions = torch.max(outputs.data, dim = 1)
        acc = calcuate_accuracy(predictions, targets)

        # loss = criterion(predictions, targets)
        # epoch_loss += loss.item()

        no_of_iterations += 1
        no_of_examples += targets.size(0)
                
        metric_acc.update(predictions, targets)
        metric_f1.update(outputs, targets)
        metric_f1_micro.update(outputs, targets)
        metric_f1_macro.update(outputs, targets)
        metric_precision.update(predictions, targets)
        metric_recall.update(predictions, targets)

        optimizer.zero_grad()
        loss.backward()
        # for GPU
        optimizer.step()

        Logger.current_logger().report_scalar(
            "train", "loss", iteration = (epoch * len(iterator) + batch_idx), value = loss.item())
#----
        # print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #         epoch, batch_idx * len(batch['ids']), len(iterator),
        #         100. * batch_idx / len(iterator), loss.item()))
        

    epoch_loss = epoch_loss/no_of_iterations
    epoch_acc = (acc*100)/no_of_examples
        
    acc_torch = metric_acc.compute()
    print(f"Training Accuracy: {acc_torch}")
    
    f1 = metric_f1.compute()
    print(f"Training F1: {f1}")
    
    f1_micro = metric_f1_micro.compute()
    print(f"Training F1 Micro: {f1_micro}")

    f1_macro = metric_f1_macro.compute()
    print(f"Training F1 Macro: {f1_macro}")
 
    precision = metric_precision.compute()
    print(f"Training Precision: {precision}")

    recall = metric_recall.compute()
    print(f"Training Recall: {recall}")
    
    print(f"Training Loss Epoch: {epoch_loss}")
  
    Logger.current_logger().report_scalar(
        "train", "accuracy", iteration=epoch, value=acc_torch)

    
    metric_acc.reset()
    metric_f1.reset()
    metric_f1_micro.reset()
    metric_f1_macro.reset()
    metric_precision.reset()
    metric_recall.reset()
    
    return epoch_loss, epoch_acc

# evaluation routine
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    no_of_iterations = 0
    no_of_examples = 0    
    
   # torch metrics
    metric_acc = torchmetrics.Accuracy().to(torch.device("cuda", 0))
    metric_f1 = torchmetrics.F1(num_classes = 2, average="none").to(torch.device("cuda", 0))
    metric_f1_micro = torchmetrics.F1(num_classes = 2).to(torch.device("cuda", 0))
    metric_f1_macro = torchmetrics.F1(num_classes = 2, average='macro').to(torch.device("cuda", 0))
    metric_precision = torchmetrics.Precision(num_classes = 2, average="none").to(torch.device("cuda", 0))
    metric_recall = torchmetrics.Recall(num_classes = 2, average="none").to(torch.device("cuda", 0))
  

    model.eval()
    
    with torch.no_grad():
    
        for _, batch in tqdm(enumerate(iterator, 0)):

            ids = batch['ids'].to(DEVICE, dtype = torch.long)
            mask = batch['mask'].to(DEVICE, dtype = torch.long)
            token_type_ids = batch['token_type_ids'].to(DEVICE, dtype = torch.long)
            targets = batch['targets'].to(DEVICE, dtype = torch.long)

            outputs = model(ids, mask)
        
            _, predictions = torch.max(outputs.data, dim = 1)

            loss = criterion(outputs, targets)
            
            acc = calcuate_accuracy(predictions, targets)

            epoch_loss += loss.item()
    
            metric_acc.update(predictions, targets)
            metric_f1.update(outputs, targets)
            metric_f1_micro.update(outputs, targets)
            metric_f1_macro.update(outputs, targets)
            metric_precision.update(predictions, targets)
            metric_recall.update(predictions, targets)

            no_of_iterations += 1
            no_of_examples += targets.size(0)
    
    epoch_loss = epoch_loss/no_of_iterations
    epoch_acc = (acc*100)/no_of_iterations #no_of_examples

    # print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    #     epoch_loss, acc, len(iterator),
    #     100. * acc / len(iterator)))

    acc_torch = metric_acc.compute()
    print(f"Validation Accuracy: {acc_torch}")
    
    f1 = metric_f1.compute()
    print(f"Validation F1 Validation: {f1}")
    
    f1_micro = metric_f1_micro.compute()
    print(f"Validation F1 Micro: {f1_micro}")

    f1_macro = metric_f1_macro.compute()
    print(f"Validation F1 Macro: {f1_macro}")
 
    precision = metric_precision.compute()
    print(f"Validation Precision: {precision}")

    recall = metric_recall.compute()
    print(f"Validation Recall: {recall}")
    
    print(f"Validation Loss Epoch: {epoch_loss}")

    # clear ml
    Logger.current_logger().report_scalar(
        "test", "loss", iteration=epoch, value=epoch_loss)
    Logger.current_logger().report_scalar(
        "test", "accuracy", iteration=epoch, value=acc_torch)

    metric_acc.reset()
    metric_f1.reset()
    metric_f1_micro.reset()
    metric_f1_macro.reset()
    metric_precision.reset()
    metric_recall.reset()
             
    return epoch_loss, epoch_acc


# experiment loop
for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, cross_entropy_loss)
    valid_loss, valid_acc = evaluate(model, valid_iterator, cross_entropy_loss)
        
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


test_loss, test_acc = evaluate(model, test_iterator, cross_entropy_loss)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

task.close()

0it [00:00, ?it/s]/home/tegzes/anaconda3/envs/ml/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
2219it [10:43,  3.45it/s]


Training Accuracy: 0.7458314299583435
Training F1: tensor([0.8537, 0.0309], device='cuda:0')
Training F1 Micro: 0.7458314299583435
Training F1 Macro: 0.4423311650753021
Training Precision: tensor([0.7478, 0.5000], device='cuda:0')
Training Recall: tensor([0.9946, 0.0160], device='cuda:0')
Training Loss Epoch: 0.5568178544503416


555it [00:31, 17.90it/s]


Validation Accuracy: 0.745945930480957
Validation F1 Validation: tensor([0.8505, 0.1557], device='cuda:0')
Validation F1 Micro: 0.745945930480957
Validation F1 Macro: 0.5030829310417175
Validation Precision: tensor([0.7624, 0.4483], device='cuda:0')
Validation Recall: tensor([0.9616, 0.0942], device='cuda:0')
Validation Loss Epoch: 0.5333852529257267
Epoch: 01
	Train Loss: 0.557 | Train Acc: 0.00%
	 Val. Loss: 0.533 |  Val. Acc: 18.02%


2219it [10:37,  3.48it/s]


...

Training Accuracy: 0.9504281282424927
Training F1: tensor([0.9668, 0.9018], device='cuda:0')
Training F1 Micro: 0.9504281282424927
Training F1 Macro: 0.9343165755271912
Training Precision: tensor([0.9645, 0.9083], device='cuda:0')
Training Recall: tensor([0.9692, 0.8954], device='cuda:0')
Training Loss Epoch: 0.14204120561662045


555it [00:30, 17.99it/s]


Validation Accuracy: 0.73153156042099
Validation F1 Validation: tensor([0.8285, 0.3817], device='cuda:0')
Validation F1 Micro: 0.73153156042099
Validation F1 Macro: 0.6051406264305115
Validation Precision: tensor([0.7965, 0.4466], device='cuda:0')
Validation Recall: tensor([0.8633, 0.3333], device='cuda:0')
Validation Loss Epoch: 0.8714719046582677
Epoch: 03
	Train Loss: 0.142 | Train Acc: 4.51%
	 Val. Loss: 0.871 |  Val. Acc: 0.00%


694it [00:38, 18.18it/s]

Validation Accuracy: 0.7420749068260193
Validation F1 Validation: tensor([0.8397, 0.3395], device='cuda:0')
Validation F1 Micro: 0.7420749068260193
Validation F1 Macro: 0.5896163582801819
Validation Precision: tensor([0.7976, 0.4340], device='cuda:0')
Validation Recall: tensor([0.8866, 0.2788], device='cuda:0')
Validation Loss Epoch: 0.8915217733830925
Test Loss: 0.892 | Test Acc: 14.41%
